In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))
from policy import ConvNet, ResNet
from concepts import DynamicConcepts, linear_regression
import absl.logging
import json
import string
import random
import numpy as np
import os
from env import gogame
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
import tensorflow as tf

absl.logging.set_verbosity(absl.logging.ERROR)

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

session_name = "falcon"
model_name = "net"
board_size = 5
board_name = f'{board_size}x{board_size}'
agents_to_sample = [0, 10, 20, 60, 100, 500, 1000, 3000]
resnet = True
random_subpar_rollout = True

model_type = "resnet" if resnet else "convnet"

# Mcts simulations
simulations = 1000

full_model_path = f"../models/saved_sessions/{model_type}/board_size_{board_size}/{session_name}/"

mcts_model_path = f"../models/best_models/{model_type}/board_size_{board_size}/net_3000.keras"

init_state, concept_type_single = DynamicConcepts.opening_play(board_size)

print("Initial state:\n", gogame.str(init_state))

dynamic_concept = DynamicConcepts(init_state, simulations, board_size, concept_type_single, mcts_model_path, random_subpar_rollout, resnet)

CONCEPT_NAME = DynamicConcepts.opening_play.__name__

Initial state:
 	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 0, White Area: 0



2024-02-13 13:49:42.898241: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-13 13:49:42.898628: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmptp6uvlr5/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmptp6uvlr5/assets
2024-02-13 13:49:53.767556: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmptp6uvlr5
2024-02-13 13:49:53.774809: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-13 13:49:53.774831: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmptp6uvlr5
2024-02-13 13:49:53.803610: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-13 13:49:53.985137: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmptp6uvlr5
2024-02-13 13:49:54.025291: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 257741 microseconds.
INFO: Cre

In [4]:
positive_cases = []
negative_cases = []

pos, neg = dynamic_concept.generate_cases()

positive_cases.extend(pos)
negative_cases.extend(neg)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [5]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (11, 5, 5, 5)
Negative cases:  (14, 5, 5, 5)


In [236]:
# If larger than 50, do not visualize the tree do to the size (crashes the kernel)
if simulations <= 100:
    graph = dynamic_concept.view_tree()

    graph.render('./visualization/images/tree', view=True)

In [36]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [215]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  0


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [ ]:
# Print all the negative cases
for i in range(len(negative_cases)):
    print(f"Negative case {i}:")
    print(negative_cases[i])
    print(gogame.str(negative_cases[i], nn_format=True))
    print()

In [ ]:
# Probing
BINARY = True
epochs = 100

In [8]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

POSITIONS_TO_CONSIDER = points.shape[0]

print("Positions to consider: ", POSITIONS_TO_CONSIDER)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=True,
    epochs=50,
    dynamic=True
)

print("Regression score: ", score)

score = 0 if score < 0 else score

# Remove the files if they exist
if os.path.exists(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/"):
    # Test if epoch folder exists
    if os.path.exists(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}"):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}"):
            os.remove(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}/{file}")

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/dynamic", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}", exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}/{random_suffix}.json", "w") as f:
    json.dump(score, f)

Positions to consider:  25
Epoch 1/50


2024-02-13 13:57:20.278389: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 13:57:20.316106: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 820ms/step - loss: 0.8422
Epoch 2/50
1/1 [==============================] - 0s 44ms/step - loss: 0.8420
Epoch 3/50
1/1 [==============================] - 0s 40ms/step - loss: 0.8418
Epoch 4/50
1/1 [==============================] - 0s 34ms/step - loss: 0.8416
Epoch 5/50
1/1 [==============================] - 0s 27ms/step - loss: 0.8414
Epoch 6/50
1/1 [==============================] - 0s 24ms/step - loss: 0.8412
Epoch 7/50
1/1 [==============================] - 0s 41ms/step - loss: 0.8411
Epoch 8/50
1/1 [==============================] - 0s 31ms/step - loss: 0.8409
Epoch 9/50
1/1 [==============================] - 0s 44ms/step - loss: 0.8407
Epoch 10/50
1/1 [==============================] - 0s 31ms/step - loss: 0.8405
Epoch 11/50
1/1 [==============================] - 0s 46ms/step - loss: 0.8403
Epoch 12/50
1/1 [==============================] - 0s 31ms/step - loss: 0.8402
Epoch 13/50
1/1 [==============================] - 0s 29ms/step - loss:

2024-02-13 13:57:22.301828: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [10]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, path)
    else:
        model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=100,
            dynamic=True
        )
        score = 0 if score < 0 else score
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}"):
        # Test if epoch folder exists
        if os.path.exists(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}"):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}"):
                os.remove(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}/{file}")

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/dynamic", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}", exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}/{random_suffix}.json", "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)

Getting activation outputs: 100%|██████████| 1/1 [00:00<00:00, 22.03it/s]

Performing regression for layer 0
Epoch 1/100



2024-02-13 14:03:18.783245: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 14:03:18.819569: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 765ms/step - loss: 1.1926
Epoch 2/100
1/1 [==============================] - 0s 50ms/step - loss: 1.1909
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1891
Epoch 4/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1874
Epoch 5/100
1/1 [==============================] - 0s 48ms/step - loss: 1.1857
Epoch 6/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1840
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1823
Epoch 8/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1806
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1789
Epoch 10/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1772
Epoch 11/100
1/1 [==============================] - 0s 45ms/step - loss: 1.1755
Epoch 12/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1738
Epoch 13/100
1/1 [==============================] - 0s 23ms/

2024-02-13 14:03:21.891696: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 14:03:22.264783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 14:03:22.297875: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

2024-02-13 14:07:09.499411: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 14:07:09.889529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 14:07:09.922086: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 710ms/step - loss: 1.1820
Epoch 2/100
1/1 [==============================] - 0s 37ms/step - loss: 1.1802
Epoch 3/100
1/1 [==============================] - 0s 60ms/step - loss: 1.1784
Epoch 4/100
1/1 [==============================] - 0s 56ms/step - loss: 1.1766
Epoch 5/100
1/1 [==============================] - 0s 40ms/step - loss: 1.1748
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1731
Epoch 7/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1713
Epoch 8/100
1/1 [==============================] - 0s 39ms/step - loss: 1.1695
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1677
Epoch 10/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1660
Epoch 11/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1642
Epoch 12/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1624
Epoch 13/100
1/1 [==============================] - 0s 26ms/

2024-02-13 14:07:13.071725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 14:07:13.423325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 14:07:13.455464: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 719ms/step - loss: 0.7678
Epoch 2/100
1/1 [==============================] - 0s 57ms/step - loss: 0.7678
Epoch 3/100
1/1 [==============================] - 0s 50ms/step - loss: 0.7677
Epoch 4/100
1/1 [==============================] - 0s 50ms/step - loss: 0.7677
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 0.7676
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 0.7676
Epoch 7/100
1/1 [==============================] - 0s 53ms/step - loss: 0.7675
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 0.7675
Epoch 9/100
1/1 [==============================] - 0s 24ms/step - loss: 0.7674
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 0.7674
Epoch 11/100
1/1 [==============================] - 0s 48ms/step - loss: 0.7673
Epoch 12/100
1/1 [==============================] - 0s 23ms/step - loss: 0.7673
Epoch 13/100
1/1 [==============================] - 0s 23ms/

2024-02-13 14:07:16.569683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
